In [80]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.base import BaseEstimator 
import warnings 
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')

In [66]:
data = pd.read_parquet('/Users/ilagulakin/Desktop/AI & code/cups/VSEROS AI 25/MainPart/B/train_data.pq')
ss = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/cups/VSEROS AI 25/MainPart/B/sample_submission.csv')
solution = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/cups/VSEROS AI 25/MainPart/B/weighted_submission.csv')

In [98]:
ss_co = ss = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/cups/VSEROS AI 25/MainPart/B/sample_submission.csv')

In [67]:
class WeightedRecommender(BaseEstimator): 
    def __init__(self, topn, k): 
        self.k = k 
        self.topn = topn 

    def fit(self, train): 
        self.last_date = train.date.max()  
        
        self.global_pop = (
            data[data.date == last_date].item_id.value_counts().head(100).index.to_list()
        )

        recent = data[data.date >= (self.last_date - self.k + 1)] 
        recent['weight'] = (self.k - (self.last_date - recent.date)) / self.k 

        scores = (
            recent 
            .groupby(['user_id', 'item_id'])['weight']
            .sum()
            .reset_index() 
            .sort_values(['user_id', 'weight'], ascending=[True, False]) 
        ) 

        self.user_items = (
            scores 
            .groupby(['user_id', 'item_id']) 
            .head(self.topn)
            .groupby('user_id')['item_id'] 
            .apply(list) 
            .to_dict() 
        )

    def predict(self, user_id): 
        recs = [] 
        used = set() 

        for item in self.user_items.get(user_id, []): 
            if len(recs) >= self.topn: 
                return recs 
            recs.append(item) 
            used.add(item) 

        for glob in self.global_pop: 
            if len(recs) == self.topn: 
                break 

            if glob not in used: 
                recs.append(glob)

        return recs  

In [68]:
class Config: 
    topn = 20 
    k = 18

In [69]:
recsys = WeightedRecommender(Config.topn, Config.k)

In [70]:
recsys.fit(train) 

In [73]:
predicts = {}
for user_id in ss.user_id.unique(): 
    predicts[user_id] = recsys.predict(user_id)

In [86]:
for idx, row in tqdm(ss.iterrows(), total=len(ss)): 
    group_idx = idx%len(predicts[row['user_id']])
    ss.loc[idx, 'item_id'] = predicts[row['user_id']][group_idx]

  0%|          | 0/5864600 [00:00<?, ?it/s]

In [88]:
ss.to_csv('prediction.csv')